## Transformers库
情感分析教程来源于 https://huggingface.co/docs/transformers/quicktour 。
### 一、使用pipeline
第一次使用pipeline载入任务时，如"sentiment-analysis"，预训练模型和分词器会被自动下载，pipeline将预训练模型与分词器结合起来，需要导入两个类，分别是AutoTokenizer——用于下载分词器，以及AutoModelForSequenceClassification——用于下载模型。

In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline
from transformers import BertTokenizer,BertModel

使用from_pretrained方法来加载训练过的模型（由transformers社区提供）。

In [2]:
model_name='distilbert-base-uncased-finetuned-sst-2-english'
model=AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name)
classifier=pipeline('sentiment-analysis',model=model,tokenizer=tokenizer)

In [3]:
results=classifier(['Happy!','I hate you!'])
results

[{'label': 'POSITIVE', 'score': 0.9998652935028076},
 {'label': 'NEGATIVE', 'score': 0.9987472295761108}]

### 二、使用Tokenizer、Model、Configuration
因为分词的方法众多，故Model与Tokennizer所使用的模型必须一致。Tokenizer主要包含两个功能：
1. 依据某种分词规则，将给定的一句话划分为单词（token），分词规则由使用的模型决定；
2. 将token转换成数字（词向量），并构建张量，主要通过vocab来实现——包含在from_pretrained方法中。

Model接收Tokenizer的输出，Configuration用于自定义超参数。

In [4]:
inputs=tokenizer('Hello world!')

tokenizer返回了一个字典，input_ids表示每个token在vocab中所对应的位置，attention_mask能够帮助模型更好的理解句子的含义。

In [5]:
inputs

{'input_ids': [101, 7592, 2088, 999, 102], 'attention_mask': [1, 1, 1, 1, 1]}

也可以向tokenizer传入由句子组成的list，并设置多种参数：
1. padding=True，对齐词向量的长度；
2. trunction=True，规定词向量的最大长度，需同时设置max_length；
3. return_tensors='pt'，返回pytorch的数据类型。

In [6]:
batch=tokenizer(
    ["On one side of the border between Russia and Ukraine, more than 100,000 of Moscow's troops are massed.",'On the southern side, thousands of Ukrainian citizens are getting military training to repel what western intelligence says is a possible Russian attack.'],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors='pt'
)
batch.items()

dict_items([('input_ids', tensor([[  101,  2006,  2028,  2217,  1997,  1996,  3675,  2090,  3607,  1998,
          5924,  1010,  2062,  2084,  2531,  1010,  2199,  1997,  4924,  1005,
          1055,  3629,  2024,  3742,  2098,  1012,   102,     0],
        [  101,  2006,  1996,  2670,  2217,  1010,  5190,  1997,  5969,  4480,
          2024,  2893,  2510,  2731,  2000, 16360,  2884,  2054,  2530,  4454,
          2758,  2003,  1037,  2825,  2845,  2886,  1012,   102]])), ('attention_mask', tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]))])

接下来便是将词向量送入model中，对于pytorch模型，需要解包字典，即添加**

In [7]:
output=model(**batch)
output

SequenceClassifierOutput([('logits', tensor([[-0.6354,  0.7732],
                                   [ 1.8996, -1.6244]], grad_fn=<AddmmBackward0>))])

所有的Transformers模型的返回值均未施加激活函数，故需手动施加。

In [8]:
import torch.nn.functional as F
activation=F.softmax(output.logits,dim=1)
activation

tensor([[0.1965, 0.8035],
        [0.9714, 0.0286]], grad_fn=<SoftmaxBackward0>)

对于监督学习，模型能够计算出损失值。

In [9]:
import torch
output=model(**batch,labels=torch.tensor([1,0]))
output

SequenceClassifierOutput([('loss', tensor(0.1239, grad_fn=<NllLossBackward0>)),
                          ('logits', tensor([[-0.6354,  0.7732],
                                   [ 1.8996, -1.6244]], grad_fn=<AddmmBackward0>))])

### 三、文本分类实战

In [10]:
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset#Dataset与DataLoader配合使用，用Dataset读取数据，再用DataLoader自定义batch
from transformers import BertTokenizer,BertForSequenceClassification,BertConfig
from transformers import AdamW
import torch
import pandas as pd

In [11]:
dropout=0.1
num_labels=2
lr=1e-5
weight_decay=1e-2#正则项系数之一，防止过拟合
epochs=2
batch_size=32

首先简单查看数据

In [12]:
temp=pd.read_csv('/content/drive/My Drive/Colab Notebooks/sentiment/sentiment.train.data',sep='\t',names=['text','label'])

In [13]:
temp.head()

,text,label
0,贝贝好爱干净 每天出门都要洗澡 还喜欢喝蒙牛 不喜欢蹲地方 喜欢坐凳子上还喜欢和我坐在一起~,1
1,感觉好像是文科生看一本《高等数学》的教材一样，流水账一般，只是背景很好罢了，选择在这样一个竞...,0
2,"很安静,隔音设施不错.服务员态度很好,下次还会选这里",1
3,1 感觉外观还可以，符合我的要求，体积虽不算小，但比它大的翻盖手机还是很多的。2 比一张IC...,1
4,收到后，包装完好。笔记本封条完好。 性价比很高，DVD驱动盘包含VISTA所有必备的驱动，方便。,1


In [14]:
class SentimentDataset(Dataset):
    #所定义的三个函数是必须的
    def __init__(self,path):
        self.data=pd.read_csv(path,sep='\t',names=['text','label'])
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,index):#取出指定元素
        text=self.data.loc[index,'text']
        label=self.data.loc[index,'label']
        sample={"text":text,'label':label}
        return sample

train=SentimentDataset('/content/drive/My Drive/Colab Notebooks/sentiment/sentiment.train.data')
valid=SentimentDataset('/content/drive/My Drive/Colab Notebooks/sentiment/sentiment.valid.data')

train=DataLoader(train,batch_size=batch_size,shuffle=True)
valid=DataLoader(valid,batch_size=batch_size,shuffle=True)

def getSize(dataloader):
    labels=next(iter(dataloader))['label']
    features=next(iter(dataloader))['text']
    print(f'labels size:{len(labels)},features size:{len(features)}')


In [15]:
print(next(iter(train))['text'][:5])
print(next(iter(train))['label'][:5])

['呜呜呜~~~我的蒙牛优益c啊！！好不容易买一瓶。给我撒了一半！', '我买了一套，是冲着作者买的。没什么内容和深度。', '宝贝一如既往的好，这已经是第三台了，物流也很快', '国航就爱配蒙牛酸~奶~ 如果中国有几千张这么“刺儿头”的脸，制度不灵的情况下，民航业也会改善很多......干，什么业又不是呢？ 国航搞这些小动作只能忽悠傻逼，可他是罗胖子啊，这张脸以后估计所有航空公司都会谨记了', '无聊,无病呻吟,照她的说法,女人最好赶快找个有钱人嫁了,还得赶在20几岁的时候,晕死,自己认为自己是贵族,别人就当你是贵族,有病吧,棒子就是会意淫!']
tensor([0, 1, 0, 1, 0])


In [16]:
len(train)#共有527个batch，说明样本总数为527*32

527

In [17]:
getSize(train)

labels size:32,features size:32


In [18]:
tokenizer=BertTokenizer.from_pretrained('hfl/chinese-roberta-wwm-ext')
model=BertForSequenceClassification.from_pretrained('hfl/chinese-roberta-wwm-ext')
config=BertConfig.from_pretrained('hfl/chinese-roberta-wwm-ext',num_labels=num_labels,hidden_dropout=dropout)

Some weights of the model checkpoint at hfl/chinese-roberta-wwm-ext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model che

In [19]:
optimizer=AdamW(model.parameters(),lr=lr)
criterion=nn.CrossEntropyLoss()

device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
from transformers import get_scheduler
num_training_steps=epochs*len(train)

lr_scheduler=get_scheduler('linear',optimizer=optimizer,num_warmup_steps=0,num_training_steps=num_training_steps)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [20]:
def train_loop(data,tokenizer,model,optimizer,criterion,device):
    loop_acc=0
    loop_loss=0
    model.train()
    for index,batch in enumerate(data):
        #因为原始数据中包括"text"、"label"
        label=batch['label']
        text=batch['text']
        label=label.to(device)
        #分词
        token_text=tokenizer(text,max_length=100,add_special_tokens=True,truncation=True,padding=True,return_tensors='pt')
        token_text=token_text.to(device)
        #传入模型
        output=model(**token_text,labels=label)#返回的第一个值是loss，第二个值是batch_size个logits
        prob=output[1]
        pred=prob.argmax(dim=1)
        #计算loss
        loss=criterion(prob,label)#交叉熵损失函数要求第一个输入值为预测概率矩阵，而非label
        loop_loss+=loss
        acc=(pred==label).sum()
        loop_acc+=acc
        #反向传播
        loss.backward()
        #更新权重和学习率
        optimizer.step()
        lr_scheduler.step()
        #梯度清零
        optimizer.zero_grad()
        if index == 300:
          break
        if index % 50 == 0:
            print(f'前{index+1}个batch的累计损失值为：{loop_loss}，准确率为{loop_acc/(index+1)/batch_size}')

def test_loop(data,tokenizer,model,optimizer,criterion,device):
    loop_acc=0
    loop_loss=0
    model.eval()
    with torch.no_grad():
        for index,batch in enumerate(data):
            label=batch['label']
            text=batch['text']
            label=label.to(device)
            #分词
            token_text=tokenizer(text,max_length=100,add_special_tokens=True,truncation=True,padding=True,return_tensors='pt').to(device)
            #传入模型
            output=model(**token_text,labels=label)
            prob=output[1]
            pred=prob.argmax(dim=1)
            #计算loss
            loss=criterion(prob,label)
            loop_loss+=loss
            acc=(pred==label).sum()
            loop_acc+=acc

            if index % 50 == 0:
                print(f'前{index+1}个batch的累计损失值为：{loop_loss}，准确率为{loop_acc/(index+1)/batch_size}')

In [21]:
for i in range(epochs):
    print('\n','-'*100,sep='')
    print(f'第{i+1}次结果：')
    train_loop(train,tokenizer,model,optimizer,criterion,device)
    test_loop(valid,tokenizer,model,optimizer,criterion,device)


----------------------------------------------------------------------------------------------------
第1次结果：
前1个batch的累计损失值为：0.7147435545921326，准确率为0.5
前51个batch的累计损失值为：27.39373779296875，准确率为0.7334558963775635
前101个batch的累计损失值为：40.4895133972168，准确率为0.8189975023269653
前151个batch的累计损失值为：53.313682556152344，准确率为0.8485099077224731
前201个batch的累计损失值为：64.30728912353516，准确率为0.8658270835876465
前251个batch的累计损失值为：75.95376586914062，准确率为0.8751245141029358
前1个batch的累计损失值为：0.2706177532672882，准确率为0.90625
前51个batch的累计损失值为：10.098142623901367，准确率为0.930759847164154

----------------------------------------------------------------------------------------------------
第2次结果：
前1个batch的累计损失值为：0.24100948870182037，准确率为0.875
前51个batch的累计损失值为：9.166712760925293，准确率为0.9350490570068359
前101个batch的累计损失值为：17.778465270996094，准确率为0.9396658539772034
前151个batch的累计损失值为：25.911706924438477，准确率为0.9395695328712463
前201个batch的累计损失值为：33.16047668457031，准确率为0.9426305890083313
前251个batch的累计损失值为：41.41641616821289，准确率为0.942729115486145